# Caclium imaging analysis 

In [22]:
# Enhanced Calcium Imaging Data Analysis Script
# This script analyzes calcium imaging data to produce visualizations
# with comprehensive statistical analysis and proper scientific annotations

import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
from scipy.stats import linregress, kruskal, mannwhitneyu, ttest_ind, shapiro
from datetime import datetime

# ==========================================================================
# SETUP AND CONFIGURATION
# ==========================================================================

# Set styles for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("paper", font_scale=1.2)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['svg.fonttype'] = 'none'  # Ensures text remains as text in SVG files

# Define color scheme for consistency
CONDITION_COLORS = {
    'WT': '#1f77b4',              # Blue
    'Nav1.1 activator': '#ff7f0e', # Orange
    'PTZ': '#2ca02c',              # Green
    'Veratridine': '#d62728'       # Red
}

# Default order of conditions for consistent plotting
DEFAULT_CONDITIONS = ['WT', 'Nav1.1 activator', 'PTZ', 'Veratridine']

# ==========================================================================
# DATA PROCESSING FUNCTIONS
# ==========================================================================

def calculate_kinetics_peaks(file_path):
    """
    Calculate calcium imaging kinetics from a CSV file.
    
    Args:
        file_path (str): Path to the CSV file containing calcium imaging data
        
    Returns:
        list: List of dictionaries containing kinetics for each ROI
    """
    # Read the CSV file, skipping the header row
    data = pd.read_csv(file_path, header=None, skiprows=1)
    
    # Identify frame column (first column) and ROI columns (all others)
    frame_col, roi_cols = 0, data.columns[1:]

    results = []
    # Process each ROI column separately
    for roi_col in roi_cols:
        # Extract time frames and fluorescence intensity values, dropping NaN values
        frames = data[frame_col].dropna().values
        intensity = data[roi_col].dropna().values
        
        # Skip if we don't have enough data points
        if len(frames) < 2 or len(intensity) < 2:
            continue

        # Calculate baseline as the mean intensity
        baseline = np.mean(intensity)
        
        # Skip if baseline is zero or negative (invalid data)
        if baseline <= 0:
            continue

        # Calculate ΔF/F (normalized change in fluorescence)
        dff = (intensity - baseline) / baseline
        
        # Find peaks in the signal with prominence > 0.1 and minimum distance of 2 frames
        peaks, peak_properties = find_peaks(dff, prominence=0.1, distance=2)
        
        # Calculate peak amplitude (mean of peak heights)
        amplitude = np.mean(dff[peaks]) if peaks.size else 0
        
        # Calculate slope of the overall trend line
        slope = linregress(frames, dff).slope if peaks.size else 0
        
        # Calculate full width at half maximum (FWHM) for peaks
        widths, _, _, _ = peak_widths(dff, peaks, rel_height=0.5)
        fwhm_val = np.mean(widths) if widths.size else 0
        
        # Store results in a dictionary for this ROI
        results.append({
            'NumPeaks': len(peaks),       # Number of detected peaks
            'Amplitude': amplitude,        # Mean peak amplitude
            'Slope': slope,                # Overall signal trend slope
            'FWHM': fwhm_val,              # Mean peak width at half maximum
            'IsActive': len(peaks) >= 2    # Define active cell as having 2+ peaks
        })
    
    return results

def parse_filename(filename):
    """
    Parse the filename to extract date, condition, replicate, and measurement.
    
    Args:
        filename (str): Name of the CSV file
        
    Returns:
        tuple: (date_str, condition, replicate, measurement)
    """
    # Remove .Results.csv suffix if present
    clean_filename = filename.replace('.Results.csv', '')
    
    # Define regex pattern for filename format: YYYYMMDD_Condition#.# 
    pattern = r"^(\d{8})_(WT|Nav|PTZ|Vera)(\d*)\.(\d+)"
    
    # Try to match the pattern
    match = re.match(pattern, clean_filename)
    
    if not match:
        # Return None values if pattern doesn't match
        return None, None, None, None
    
    # Extract and process components
    date_str, condition, replicate, meas = match.groups()
    
    # Convert replicate and measurement to integers
    replicate = int(replicate or 0)  # Default to 0 if replicate is empty
    meas = int(meas[0])              # Take first digit of measurement
    
    # Map condition codes to full names
    condition_map = {
        'WT': 'WT',
        'Nav': 'Nav1.1 activator',
        'PTZ': 'PTZ',
        'Vera': 'Veratridine'
    }
    
    # Return extracted metadata
    return date_str, condition_map.get(condition, condition), replicate, meas

def collect_data(folder_path):
    """
    Collect and process data from all CSV files in a folder.
    
    Args:
        folder_path (str): Path to folder containing CSV files
        
    Returns:
        DataFrame: Processed data from all files
    """
    data = []
    file_count = 0
    
    # Iterate through each file in the folder
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            # Build the full file path
            path = os.path.join(folder_path, file)
            file_count += 1
            
            # Parse the filename to extract metadata
            date, cond, rep, meas = parse_filename(file)
            
            # Skip if filename doesn't match expected pattern
            if not date:
                print(f"Skipping file with invalid name format: {file}")
                continue
                
            # Calculate kinetics for all ROIs in this file
            kinetics = calculate_kinetics_peaks(path)
            
            # Add metadata to each ROI's kinetics data
            for k in kinetics:
                k.update({
                    'Date': date,
                    'Condition': cond,
                    'Replicate': rep,
                    'Measurement': meas,
                    'File': file
                })
                data.append(k)
                
    print(f"Processed {file_count} files with {len(data)} total ROIs")
    
    # Convert to DataFrame
    return pd.DataFrame(data)

def remove_outliers(df, metric, multiplier=1.5):
    """
    Remove outliers using the Interquartile Range (IQR) method.
    
    Args:
        df (DataFrame): Input data
        metric (str): Column name to check for outliers
        multiplier (float): IQR multiplier for outlier boundary (default: 1.5)
        
    Returns:
        DataFrame: Data with outliers removed
    """
    # Calculate first quartile (Q1) and third quartile (Q3)
    Q1 = df[metric].quantile(0.25)
    Q3 = df[metric].quantile(0.75)
    
    # Calculate interquartile range
    IQR = Q3 - Q1
    
    # Define lower and upper bounds for outliers
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    # Filter DataFrame to keep only non-outlier values
    filtered_df = df[(df[metric] >= lower_bound) & (df[metric] <= upper_bound)]
    
    # Calculate how many outliers were removed
    removed_count = len(df) - len(filtered_df)
    
    if removed_count > 0:
        print(f"Removed {removed_count} outliers from {metric} ({removed_count/len(df)*100:.1f}%)")
    
    return filtered_df

def calculate_active_percentage(df):
    """
    Calculate the percentage of active cells for each condition and measurement.
    
    Args:
        df (DataFrame): Input data
        
    Returns:
        DataFrame: Percentage of active cells by condition and measurement
    """
    # Group by condition and measurement
    # First, count active cells (where IsActive is True)
    active_counts = df[df['IsActive']].groupby(['Condition', 'Measurement']).size()
    
    # Count total cells
    total_counts = df.groupby(['Condition', 'Measurement']).size()
    
    # Create a DataFrame from these counts
    result_df = pd.DataFrame({
        'TotalCells': total_counts,
        'ActiveCells': active_counts
    }).fillna(0).reset_index()  # Fill NaN with 0 for conditions with no active cells
    
    # Calculate percentage of active cells directly from the counts
    result_df['PercentActive'] = (result_df['ActiveCells'] / result_df['TotalCells'] * 100).round(1)
    
    return result_df

def perform_statistical_analysis(df, metric, conditions):
    """
    Perform comprehensive statistical analysis on a metric across conditions.
    
    Args:
        df (DataFrame): Input data
        metric (str): Metric to analyze
        conditions (list): List of conditions to compare
        
    Returns:
        dict: Statistical analysis results and summary
    """
    # Initialize results dictionary
    stats_results = {
        'metric': metric,
        'normality_tests': {},
        'normal_distribution': True,  # Assume normal until proven otherwise
        'descriptive_stats': {},
        'test_used': None,
        'overall_p_value': None,
        'pairwise_tests': [],
        'summary': []
    }
    
    # Calculate descriptive statistics for each condition
    for cond in conditions:
        cond_data = df[df['Condition'] == cond][metric]
        
        # Skip if insufficient data
        if len(cond_data) < 3:
            stats_results['descriptive_stats'][cond] = {
                'n': len(cond_data),
                'mean': cond_data.mean() if len(cond_data) > 0 else np.nan,
                'sd': cond_data.std() if len(cond_data) > 0 else np.nan,
                'median': cond_data.median() if len(cond_data) > 0 else np.nan,
                'min': cond_data.min() if len(cond_data) > 0 else np.nan,
                'max': cond_data.max() if len(cond_data) > 0 else np.nan
            }
            continue
            
        # Shapiro-Wilk test for normality (works best for n between 3 and 5000)
        w_stat, p_value = shapiro(cond_data)
        is_normal = p_value > 0.05  # p > 0.05 suggests normal distribution
        
        # Record normality test results
        stats_results['normality_tests'][cond] = {
            'w_statistic': w_stat,
            'p_value': p_value,
            'is_normal': is_normal
        }
        
        # Update overall normality flag
        if not is_normal:
            stats_results['normal_distribution'] = False
            
        # Calculate descriptive statistics
        stats_results['descriptive_stats'][cond] = {
            'n': len(cond_data),
            'mean': cond_data.mean(),
            'sd': cond_data.std(),
            'median': cond_data.median(),
            'min': cond_data.min(),
            'max': cond_data.max()
        }
    
    # Check if we have enough data for statistical tests
    valid_conditions = [cond for cond in conditions 
                      if cond in stats_results['descriptive_stats'] 
                      and stats_results['descriptive_stats'][cond]['n'] >= 3]
    
    if len(valid_conditions) < 2:
        stats_results['summary'].append("Insufficient data for statistical comparison")
        return stats_results
    
    # Choose the appropriate statistical test based on normality
    if stats_results['normal_distribution']:
        # For normal distributions, use t-tests for pairwise comparisons
        stats_results['test_used'] = "t-test (normal distribution)"
    else:
        # For non-normal distributions, use non-parametric tests
        stats_results['test_used'] = "Mann-Whitney U test (non-normal distribution)"
        
        # Perform Kruskal-Wallis test if we have more than 2 conditions
        if len(valid_conditions) > 2:
            kw_stat, kw_p = kruskal(*(df[df['Condition'] == cond][metric] for cond in valid_conditions))
            stats_results['overall_p_value'] = kw_p
            
            stats_results['summary'].append(f"Kruskal-Wallis test: H={kw_stat:.3f}, p={kw_p:.4f}")
            if kw_p < 0.05:
                stats_results['summary'].append("Significant differences found between groups (p < 0.05)")
            else:
                stats_results['summary'].append("No significant differences found between groups (p ≥ 0.05)")
    
    # Perform pairwise comparisons
    for i, cond1 in enumerate(valid_conditions):
        for cond2 in valid_conditions[i+1:]:
            data1 = df[df['Condition'] == cond1][metric]
            data2 = df[df['Condition'] == cond2][metric]
            
            if stats_results['normal_distribution']:
                # Use t-test for normal data
                stat, p_val = ttest_ind(data1, data2)
                test_name = "t-test"
            else:
                # Use Mann-Whitney U test for non-normal data
                stat, p_val = mannwhitneyu(data1, data2)
                test_name = "Mann-Whitney U"
                
            # Determine significance and record result
            significant = p_val < 0.05
            
            # Add to pairwise test results
            stats_results['pairwise_tests'].append({
                'group1': cond1,
                'group2': cond2,
                'test': test_name,
                'statistic': stat,
                'p_value': p_val,
                'significant': significant
            })
    
    # Create summary of pairwise comparisons
    for test in stats_results['pairwise_tests']:
        sig_symbol = "*" if test['significant'] else "ns"
        stats_results['summary'].append(
            f"{test['group1']} vs {test['group2']}: {test['test']}, "
            f"p={test['p_value']:.4f} ({sig_symbol})"
        )
    
    return stats_results

# ==========================================================================
# FILE AND FOLDER MANAGEMENT
# ==========================================================================

def create_data_folders(output_folder):
    """
    Create folder structure for outputs.
    
    Args:
        output_folder (str): Base output folder
        
    Returns:
        dict: Dictionary with paths to each folder
    """
    folders = {
        'active': os.path.join(output_folder, 'active_cells'),
        'boxplots': os.path.join(output_folder, 'boxplots'),
        'violin': os.path.join(output_folder, 'violin_plots'),
        'data': os.path.join(output_folder, 'processed_data'),
        'merged': os.path.join(output_folder, 'merged_figures'),
        'stats': os.path.join(output_folder, 'statistics')
    }
    
    # Create each folder
    for folder in folders.values():
        os.makedirs(folder, exist_ok=True)
        
    return folders

def generate_stats_report(stats_results, output_path):
    """
    Generate a detailed statistical report in text format.
    
    Args:
        stats_results (dict): Statistical analysis results
        output_path (str): Path to save the report
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        # Write title
        f.write(f"Statistical Analysis Report for {stats_results['metric']}\n")
        f.write("="*50 + "\n\n")
        
        # Write descriptive statistics
        f.write("Descriptive Statistics:\n")
        f.write("-"*50 + "\n")
        
        # Create a table header
        f.write(f"{'Condition':<20} {'N':<6} {'Mean':<10} {'SD':<10} {'Median':<10} {'Min':<10} {'Max':<10}\n")
        
        # Write data for each condition
        for cond, stats in stats_results['descriptive_stats'].items():
            f.write(f"{cond:<20} {stats['n']:<6} {stats['mean']:.4f} {stats['sd']:.4f} {stats['median']:.4f} {stats['min']:.4f} {stats['max']:.4f}\n")
        
        f.write("\n")
        
        # Write normality test results
        f.write("Normality Test Results (Shapiro-Wilk):\n")
        f.write("-"*50 + "\n")
        
        for cond, test in stats_results['normality_tests'].items():
            normal_status = "Normal" if test['is_normal'] else "Non-normal"
            f.write(f"{cond}: W={test['w_statistic']:.4f}, p={test['p_value']:.4f} ({normal_status})\n")
        
        f.write("\n")
        
        # Write the chosen statistical test and reasoning
        f.write("Statistical Test Selection:\n")
        f.write("-"*50 + "\n")
        
        if stats_results['normal_distribution']:
            f.write("Data appears to be normally distributed (Shapiro-Wilk p>0.05).\n")
            f.write("Using parametric tests (t-test) for comparison.\n")
        else:
            f.write("Data does not appear to be normally distributed (Shapiro-Wilk p<0.05).\n")
            f.write("Using non-parametric tests (Mann-Whitney U test) for comparison.\n")
        
        f.write("\n")
        
        # Write overall test result if available
        if stats_results['overall_p_value'] is not None:
            f.write("Overall Comparison:\n")
            f.write("-"*50 + "\n")
            f.write(f"Kruskal-Wallis test p-value: {stats_results['overall_p_value']:.4f}\n")
            if stats_results['overall_p_value'] < 0.05:
                f.write("There are significant differences between groups (p < 0.05).\n")
            else:
                f.write("No significant differences detected between groups (p >= 0.05).\n")
            f.write("\n")
        
        # Write pairwise comparisons
        if stats_results['pairwise_tests']:
            f.write("Pairwise Comparisons:\n")
            f.write("-"*50 + "\n")
            
            for test in stats_results['pairwise_tests']:
                sig_text = "Significant (p < 0.05)" if test['significant'] else "Not significant (p >= 0.05)"
                f.write(f"{test['group1']} vs {test['group2']}:\n")
                f.write(f"  Test: {test['test']}\n")
                f.write(f"  Statistic: {test['statistic']:.4f}\n")
                f.write(f"  p-value: {test['p_value']:.4f}\n")
                f.write(f"  Result: {sig_text}\n\n")
        
        # Write summary
        f.write("Summary:\n")
        f.write("-"*50 + "\n")
        for line in stats_results['summary']:
            f.write(line + "\n")

# ==========================================================================
# VISUALIZATION FUNCTIONS
# ==========================================================================

def add_significance_bars(ax, significant_pairs, y_max, y_range):
    """
    Add significance bars and annotations to a plot.
    
    Args:
        ax (Axes): Matplotlib axis object
        significant_pairs (list): List of tuples with (idx1, idx2, label)
        y_max (float): Maximum y value in the data
        y_range (float): Range of y values
        
    Returns:
        float: New y_max value after adding bars
    """
    # Height of significance bars
    bar_height = y_range * 0.05
    
    # Add bars and annotations for significant differences
    for i, (idx1, idx2, label) in enumerate(significant_pairs):
        # Calculate y position for this bar (stack them if multiple)
        y_pos = y_max + (i + 1) * bar_height * 1.5
        
        # Draw the bar
        ax.plot([idx1, idx2], [y_pos, y_pos], 'k-', lw=1.5)
        
        # Add vertical ticks at each end
        ax.plot([idx1, idx1], [y_pos - bar_height/2, y_pos], 'k-', lw=1.5)
        ax.plot([idx2, idx2], [y_pos - bar_height/2, y_pos], 'k-', lw=1.5)
        
        # Add text
        ax.text((idx1 + idx2) / 2, y_pos + bar_height/2, label, 
                ha='center', va='bottom', fontsize=9)
    
    # If we have significance bars, adjust the y-limit
    if significant_pairs:
        new_y_max = y_max + (len(significant_pairs) + 1) * bar_height * 2
        ax.set_ylim(top=new_y_max)
        return new_y_max
    
    return y_max

def add_n_annotations(ax, df_clean, metric, conditions, position='bottom'):
    """
    Add sample size annotations to a plot.
    
    Args:
        ax (Axes): Matplotlib axis object
        df_clean (DataFrame): Data for plotting
        metric (str): Metric being plotted
        conditions (list): List of conditions
        position (str): Position for annotations ('bottom', 'violin', or 'boxplot')
    """
    # Get y-axis limits and range
    ylim = ax.get_ylim()
    y_range = ylim[1] - ylim[0]
    
    for i, cond in enumerate(conditions):
        if cond in df_clean['Condition'].unique():
            n = len(df_clean[df_clean['Condition'] == cond])
            
            if position == 'bottom':
                # Position at a fixed distance from bottom of plot
                y_pos = ylim[0] + y_range * 0.05
                ax.annotate(
                    f"N={n}",
                    xy=(i, y_pos),
                    horizontalalignment='center',
                    verticalalignment='bottom',
                    fontsize=8,
                    fontweight='bold',
                    color='black',
                    alpha=0.7
                )
            
            elif position == 'violin':
                # Position relative to violin plot structure
                try:
                    violin_stats = ax.collections[i].get_paths()[0].vertices
                    violin_min = min(violin_stats[:, 1])
                    violin_range = max(violin_stats[:, 1]) - violin_min
                    y_pos = violin_min + violin_range * 0.15
                    
                    ax.annotate(
                        f"N={n}",
                        xy=(i, y_pos),
                        horizontalalignment='center',
                        verticalalignment='center',
                        fontsize=8,
                        color='black',
                        alpha=0.8,
                        weight='bold'
                    )
                except (IndexError, AttributeError):
                    # Fallback if violin structure can't be accessed
                    y_pos = ylim[0] + y_range * 0.05
                    ax.annotate(
                        f"N={n}",
                        xy=(i, y_pos),
                        horizontalalignment='center',
                        verticalalignment='bottom',
                        fontsize=8,
                        fontweight='bold',
                        color='black',
                        alpha=0.7
                    )
            
            elif position == 'boxplot':
                # Get condition-specific data
                min_val = df_clean[df_clean['Condition'] == cond][metric].min()
                y_pos = min_val + y_range * 0.05
                
                ax.annotate(
                    f"N={n}",
                    xy=(i, y_pos),
                    horizontalalignment='center',
                    verticalalignment='bottom',
                    fontsize=8,
                    color='black',
                    alpha=0.7
                )

def plot_active_percentage(active_df, output_folder, conditions=None):
    """
    Create bar plots showing percentage of active cells by condition.
    
    Args:
        active_df (DataFrame): Data containing percentage of active cells
        output_folder (str): Folder to save plots
        conditions (list, optional): List of conditions to include
        
    Returns:
        dict: Dictionary with measurement data for merged plots
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
        
    # Store data for merged plots
    meas_data = {}
    
    # Create a separate plot for each measurement
    for meas in sorted(active_df['Measurement'].unique()):
        # Filter data for this measurement
        df_meas = active_df[active_df['Measurement'] == meas].copy()
        meas_data[meas] = df_meas  # Store for merged plots
        
        # Create figure
        plt.figure(figsize=(10, 6))
        
        # Create bar plot with improved styling
        ax = sns.barplot(
            x='Condition', 
            y='PercentActive', 
            data=df_meas,
            palette=CONDITION_COLORS,
            order=conditions,
            errorbar=None
        )
        
        # Add value labels on top of bars with N values
        for i, cond in enumerate(conditions):
            if cond in df_meas['Condition'].values:
                # Get condition-specific data
                group_data = df_meas[df_meas['Condition'] == cond]
                total_cells = group_data['TotalCells'].values[0]
                percent_active = group_data['PercentActive'].values[0]
                
                # Get the corresponding bar
                if i < len(ax.patches):
                    p = ax.patches[i]
                    
                    # Add percentage value on top of bar
                    ax.annotate(
                        f"{percent_active:.1f}%",
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', 
                        va='bottom',
                        fontsize=10,
                        color='black'
                    )
                    
                    # Add N value on the middle of the bar
                    ax.annotate(
                        f"N={total_cells}",
                        (p.get_x() + p.get_width() / 2., p.get_height() / 2),
                        ha='center',
                        va='center',
                        fontsize=9,
                        color='white',
                        weight='bold'
                    )
        
        # Set labels and title
        plt.title(f'Percentage of Active Cells (Measurement {meas})', fontsize=14, pad=20)
        plt.xlabel('')
        plt.ylabel('Percentage of Active Cells (%)', fontsize=12)
        
        # Set consistent y-axis limits
        plt.ylim(0, 60)  # Adjust as needed based on your data
        
        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        # Save figures
        plt.savefig(os.path.join(output_folder, f'PercentActive_Meas_{meas}.png'), dpi=300)
        plt.savefig(os.path.join(output_folder, f'PercentActive_Meas_{meas}.svg'), format='svg')
        plt.close()
    
    return meas_data

def create_merged_active_cells_plot(active_data, output_folder, conditions=None):
    """
    Create a merged figure comparing percentage of active cells across measurements.
    
    Args:
        active_data (dict): Dictionary with active cells data by measurement
        output_folder (str): Folder to save the merged plot
        conditions (list, optional): List of conditions to include
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
        
    # Skip if we don't have data for at least two measurements
    if len(active_data) < 2:
        print("Not enough measurements for merged active cells plot")
        return
    
    # Set figure size constants
    MERGED_FIGURE_SIZE = (16, 12)
    
    # Create figure with two subplots side by side
    fig, axes = plt.subplots(1, len(active_data), figsize=MERGED_FIGURE_SIZE, sharey=True)
    
    # Set suptitle
    fig.suptitle('Percentage of Active Cells Across Measurements', fontsize=16, y=0.98)
    
    # Plot each measurement
    for i, (meas, data) in enumerate(sorted(active_data.items())):
        ax = axes[i]
        
        # Create bar plot for this measurement
        sns.barplot(
            x='Condition', 
            y='PercentActive', 
            data=data,
            palette=CONDITION_COLORS,
            order=conditions,
            errorbar=None,
            ax=ax
        )
        
        # Add value labels on top of bars with N values
        for j, cond in enumerate(conditions):
            if cond in data['Condition'].values:
                # Get condition-specific data
                group_data = data[data['Condition'] == cond]
                total_cells = group_data['TotalCells'].values[0]
                percent_active = group_data['PercentActive'].values[0]
                
                # Get the corresponding bar
                if j < len(ax.patches):
                    p = ax.patches[j]
                    
                    # Add percentage value on top of bar
                    ax.annotate(
                        f"{percent_active:.1f}%",
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', 
                        va='bottom',
                        fontsize=10,
                        color='black'
                    )
                    
                    # Add N value on# Add N value on the middle of the bar
                    ax.annotate(
                        f"N={total_cells}",
                        (p.get_x() + p.get_width() / 2., p.get_height() / 2),
                        ha='center',
                        va='center',
                        fontsize=9,
                        color='white',
                        weight='bold'
                    )
        
        # Set titles and labels
        ax.set_title(f'Measurement {meas}', fontsize=14)
        ax.set_xlabel('')
        if i == 0:  # Only set y label on the first subplot
            ax.set_ylabel('Percentage of Active Cells (%)', fontsize=12)
        else:
            ax.set_ylabel('')
        
        # Rotate x-axis labels
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
        
        # Set consistent y-axis limits
        ax.set_ylim(0, 60)  # Adjust as needed based on your data
    
    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Make room for suptitle
    
    # Save the figure
    plt.savefig(os.path.join(output_folder, 'PercentActive_Merged.png'), dpi=300)
    plt.savefig(os.path.join(output_folder, 'PercentActive_Merged.svg'), format='svg')
    plt.close()

def plot_boxplots(df, metric, output_folder, conditions=None):
    """
    Create publication-quality box plots with individual data points.
    
    Args:
        df (DataFrame): Input data
        metric (str): Metric to plot
        output_folder (str): Folder to save plots
        conditions (list, optional): List of conditions to include
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
        
    # Create a separate plot for each measurement
    for meas in sorted(df['Measurement'].unique()):
        # Filter data for this measurement and active cells
        df_meas = df[(df['Measurement'] == meas) & (df['IsActive'])]
        
        # Skip if no data
        if len(df_meas) == 0:
            print(f"No active cells for measurement {meas}, skipping boxplot")
            continue
        
        # Remove outliers for better visualization
        df_clean = remove_outliers(df_meas, metric)
        
        # Create figure
        plt.figure(figsize=(10, 6))
        
        # Create box plot with wider boxes to make room for mean lines
        ax = sns.boxplot(
            x='Condition', 
            y=metric, 
            data=df_clean,
            palette=CONDITION_COLORS,
            order=conditions,
            showfliers=False,  # Hide fliers for cleaner look
            width=0.6,         # Wider boxes
            showmeans=False    # Add custom mean lines instead
        )
        
        # Add custom mean lines that span the entire width of each box
        for i, cond in enumerate(conditions):
            if cond in df_clean['Condition'].unique():
                # Calculate mean
                mean_val = df_clean[df_clean['Condition'] == cond][metric].mean()
                
                # Use a fixed width for the mean line (0.5 on each side)
                ax.plot(
                    [i-0.3, i+0.3], 
                    [mean_val, mean_val], 
                    color='red', 
                    linewidth=3,
                    solid_capstyle='butt'
                )
        
        # Add individual data points (stripplot)
        sns.stripplot(
            x='Condition', 
            y=metric, 
            data=df_clean,
            size=4,
            alpha=0.5,
            jitter=0.2,
            color='black',
            order=conditions
        )
        
        # Perform statistical analysis
        stats_results = perform_statistical_analysis(df_clean, metric, conditions)
        
        # Add statistical annotations if significant differences found
        significant_pairs = []
        for test in stats_results['pairwise_tests']:
            if test['significant']:
                # Add pair to the list of significant pairs
                significant_pairs.append((
                    conditions.index(test['group1']),
                    conditions.index(test['group2']),
                    f"p={test['p_value']:.3f}"
                ))
        
        # Add significance bars
        y_max = df_clean[metric].max()
        y_range = df_clean[metric].max() - df_clean[metric].min()
        add_significance_bars(ax, significant_pairs, y_max, y_range)
        
        # Add sample size annotations
        add_n_annotations(ax, df_clean, metric, conditions, position='boxplot')
        
        # Set labels and title
        plt.title(f'{metric} (Measurement {meas})', fontsize=14, pad=20)
        plt.xlabel('')
        plt.ylabel(metric, fontsize=12)
        
        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')
        
        # Adjust layout and save
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{metric}_Meas_{meas}_boxplot.png'), dpi=300)
        plt.savefig(os.path.join(output_folder, f'{metric}_Meas_{meas}_boxplot.svg'), format='svg')
        plt.close()
        
        # Generate and save statistical report
        stats_path = os.path.join(output_folder, f'{metric}_Meas_{meas}_stats.txt')
        generate_stats_report(stats_results, stats_path)

def plot_violinplots(df, metric, output_folder, conditions=None):
    """
    Create publication-quality violin plots with mean indicators and significance annotations.
    
    Args:
        df (DataFrame): Input data
        metric (str): Metric to plot
        output_folder (str): Folder to save plots
        conditions (list, optional): List of conditions to include
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
        
    # Create a separate plot for each measurement
    for meas in sorted(df['Measurement'].unique()):
        # Filter data for this measurement and active cells
        df_meas = df[(df['Measurement'] == meas) & (df['IsActive'])]
        
        # Skip if no data
        if len(df_meas) == 0:
            print(f"No active cells for measurement {meas}, skipping violinplot")
            continue
        
        # Remove outliers for better visualization
        df_clean = remove_outliers(df_meas, metric)
        
        # Create figure
        plt.figure(figsize=(10, 6))
        
        # Create violin plot
        ax = sns.violinplot(
            x='Condition', 
            y=metric, 
            data=df_clean,
            palette=CONDITION_COLORS,
            order=conditions,
            inner=None,  # No inner points/box
            cut=0,       # Don't extend beyond data range
            width=0.8    # Slightly wider violins
        )
        
        # Add mean lines that span most of the violin width
        means = df_clean.groupby('Condition')[metric].mean()
        for i, cond in enumerate(conditions):
            if cond in means:
                # Make the mean line span 80% of the violin width
                plt.plot(
                    [i-0.4, i+0.4], 
                    [means[cond], means[cond]], 
                    color='red', 
                    linewidth=3
                )
        
        # Perform statistical analysis
        stats_results = perform_statistical_analysis(df_clean, metric, conditions)
        
        # Add statistical annotations if significant differences found
        significant_pairs = []
        for test in stats_results['pairwise_tests']:
            if test['significant']:
                # Add pair to the list of significant pairs
                significant_pairs.append((
                    conditions.index(test['group1']),
                    conditions.index(test['group2']),
                    f"p={test['p_value']:.3f}"
                ))
        
        # Add significance bars
        y_max = df_clean[metric].max()
        y_range = df_clean[metric].max() - df_clean[metric].min()
        add_significance_bars(ax, significant_pairs, y_max, y_range)
        
        # Add sample size annotations with positioning optimized for violin plots
        add_n_annotations(ax, df_clean, metric, conditions, position='violin')
        
        # Set labels and title
        plt.title(f'{metric} (Measurement {meas})', fontsize=14, pad=20)
        plt.xlabel('')
        plt.ylabel(metric, fontsize=12)
        
        # Rotate x-axis labels for better readability
        plt.xticks(rotation=45, ha='right')
        
        # Adjust layout and save
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{metric}_Meas_{meas}_violin.png'), dpi=300)
        plt.savefig(os.path.join(output_folder, f'{metric}_Meas_{meas}_violin.svg'), format='svg')
        plt.close()

def create_merged_boxplots_plot(df, measurement, output_folder, conditions=None):
    """
    Create a merged figure comparing all kinetics for a specific measurement using boxplots.
    
    Args:
        df (DataFrame): Full dataset with all measurements and metrics
        measurement (int): Measurement number to plot
        output_folder (str): Folder to save the merged plot
        conditions (list, optional): List of conditions to include
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
        
    # Filter for the specific measurement and active cells
    df_meas = df[(df['Measurement'] == measurement) & (df['IsActive'])]
    
    # Skip if no data
    if len(df_meas) == 0:
        print(f"No active cells for measurement {measurement}, skipping merged boxplots plot")
        return
    
    # Define metrics to plot
    metrics = ['NumPeaks', 'Amplitude', 'Slope', 'FWHM']
    
    # Set figure size constants
    MERGED_FIGURE_SIZE = (16, 12)
    
    # Create a 2x2 figure for the four metrics
    fig, axes = plt.subplots(2, 2, figsize=MERGED_FIGURE_SIZE)
    axes = axes.flatten()  # Flatten to easily iterate
    
    # Set suptitle
    fig.suptitle(f'Calcium Kinetics Boxplots (Measurement {measurement})', fontsize=16, y=0.98)
    
    # Plot each metric
    for i, metric in enumerate(metrics):
        ax = axes[i]
        
        # Remove outliers
        df_clean = remove_outliers(df_meas, metric)
        
        # Create box plot
        sns.boxplot(
            x='Condition', 
            y=metric, 
            data=df_clean,
            palette=CONDITION_COLORS,
            order=conditions,
            showfliers=False,
            width=0.6,
            ax=ax
        )
        
        # Add individual data points
        sns.stripplot(
            x='Condition', 
            y=metric, 
            data=df_clean,
            size=4,
            alpha=0.5,
            jitter=0.2,
            color='black',
            order=conditions,
            ax=ax
        )
        
        # Add custom mean lines
        for j, cond in enumerate(conditions):
            if cond in df_clean['Condition'].unique():
                mean_val = df_clean[df_clean['Condition'] == cond][metric].mean()
                ax.plot(
                    [j-0.3, j+0.3], 
                    [mean_val, mean_val], 
                    color='red', 
                    linewidth=3,
                    solid_capstyle='butt'
                )
        
        # Perform statistical analysis
        stats_results = perform_statistical_analysis(df_clean, metric, conditions)
        
        # Add statistical annotations if significant differences found
        significant_pairs = []
        for test in stats_results['pairwise_tests']:
            if test['significant']:
                # Add pair to the list of significant pairs
                significant_pairs.append((
                    conditions.index(test['group1']),
                    conditions.index(test['group2']),
                    f"p={test['p_value']:.3f}"
                ))
        
        # Add significance bars
        y_max = df_clean[metric].max()
        y_range = df_clean[metric].max() - df_clean[metric].min()
        add_significance_bars(ax, significant_pairs, y_max, y_range)
        
        # Add N annotations with improved positioning
        add_n_annotations(ax, df_clean, metric, conditions, position='bottom')
        
        # Set titles and labels
        ax.set_title(metric, fontsize=12)
        ax.set_xlabel('')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    
    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Make room for suptitle
    
    # Save the figure
    plt.savefig(os.path.join(output_folder, f'Kinetics_Meas_{measurement}_Merged_Boxplots.png'), dpi=300)
    plt.savefig(os.path.join(output_folder, f'Kinetics_Meas_{measurement}_Merged_Boxplots.svg'), format='svg')
    plt.close()

def create_merged_kinetics_plot(df, measurement, output_folder, conditions=None):
    """
    Create a merged figure comparing all kinetics for a specific measurement using violin plots.
    
    Args:
        df (DataFrame): Full dataset with all measurements and metrics
        measurement (int): Measurement number to plot
        output_folder (str): Folder to save the merged plot
        conditions (list, optional): List of conditions to include
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
        
    # Filter for the specific measurement and active cells
    df_meas = df[(df['Measurement'] == measurement) & (df['IsActive'])]
    
    # Skip if no data
    if len(df_meas) == 0:
        print(f"No active cells for measurement {measurement}, skipping merged kinetics plot")
        return
    
    # Define metrics to plot
    metrics = ['NumPeaks', 'Amplitude', 'Slope', 'FWHM']
    
    # Set figure size constants
    MERGED_FIGURE_SIZE = (16, 12)
    
    # Create a 2x2 figure for the four metrics
    fig, axes = plt.subplots(2, 2, figsize=MERGED_FIGURE_SIZE)
    axes = axes.flatten()  # Flatten to easily iterate
    
    # Set suptitle
    fig.suptitle(f'Calcium Kinetics (Measurement {measurement})', fontsize=16, y=0.98)
    
    # Plot each metric
    for i, metric in enumerate(metrics):
        ax = axes[i]
        
        # Remove outliers
        df_clean = remove_outliers(df_meas, metric)
        
        # Create violin plot
        sns.violinplot(
            x='Condition', 
            y=metric, 
            data=df_clean,
            palette=CONDITION_COLORS,
            order=conditions,
            inner=None,
            cut=0,
            width=0.8,
            scale='width',
            ax=ax
        )
        
        # Add mean lines
        for j, cond in enumerate(conditions):
            if cond in df_clean['Condition'].unique():
                mean_val = df_clean[df_clean['Condition'] == cond][metric].mean()
                ax.plot(
                    [j-0.4, j+0.4], 
                    [mean_val, mean_val], 
                    color='red', 
                    linewidth=3,
                    solid_capstyle='butt'
                )
        
        # Add N annotations with improved positioning
        add_n_annotations(ax, df_clean, metric, conditions, position='bottom')
        
        # Perform statistical analysis
        stats_results = perform_statistical_analysis(df_clean, metric, conditions)
        
        # Add statistical annotations if significant differences found
        significant_pairs = []
        for test in stats_results['pairwise_tests']:
            if test['significant']:
                # Add pair to the list of significant pairs
                significant_pairs.append((
                    conditions.index(test['group1']),
                    conditions.index(test['group2']),
                    f"p={test['p_value']:.3f}"
                ))
        
        # Add significance bars
        y_max = df_clean[metric].max()
        y_range = df_clean[metric].max() - df_clean[metric].min()
        add_significance_bars(ax, significant_pairs, y_max, y_range)
        
        # Set titles and labels
        ax.set_title(metric, fontsize=12)
        ax.set_xlabel('')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    
    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Make room for suptitle
    
    # Save the figure
    plt.savefig(os.path.join(output_folder, f'Kinetics_Meas_{measurement}_Merged.png'), dpi=300)
    plt.savefig(os.path.join(output_folder, f'Kinetics_Meas_{measurement}_Merged.svg'), format='svg')
    plt.close()

def create_merged_figures(df, active_data, output_folder, conditions=None):
    """
    Create merged figures for comprehensive comparison across measurements.
    
    Args:
        df (DataFrame): Full dataset with all measurements and metrics
        active_data (dict): Dictionary with active cells data by measurement
        output_folder (str): Folder to save merged plots
        conditions (list, optional): List of conditions to include
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
        
    # Create merged figures subfolder
    merged_folder = os.path.join(output_folder, 'merged_figures')
    os.makedirs(merged_folder, exist_ok=True)
    
    # 1. Merged % active cells figure (both measurements)
    create_merged_active_cells_plot(active_data, merged_folder, conditions)
    
    # 2. Merged kinetics figures for measurement 1 (violin plots)
    create_merged_kinetics_plot(df, 1, merged_folder, conditions)
    
    # 3. Merged kinetics figures for measurement 2 (violin plots)
    create_merged_kinetics_plot(df, 2, merged_folder, conditions)
    
    # 4. Merged boxplot figures for measurement 1
    create_merged_boxplots_plot(df, 1, merged_folder, conditions)
    
    # 5. Merged boxplot figures for measurement 2
    create_merged_boxplots_plot(df, 2, merged_folder, conditions)

# ==========================================================================
# MAIN ANALYSIS FUNCTIONS
# ==========================================================================

def create_all_figures(df, output_folder, conditions=None):
    """
    Create all required figures: active percentage, box plots, and violin plots.
    
    Args:
        df (DataFrame): Input data
        output_folder (str): Folder to save all plots
        conditions (list, optional): List of conditions to include
    """
    if conditions is None:
        conditions = DEFAULT_CONDITIONS
    
    # Define metrics to analyze
    metrics = ['NumPeaks', 'Amplitude', 'Slope', 'FWHM']
    
    # Create folder structure
    folders = create_data_folders(output_folder)
    
    # 1. Plot percentage of active cells
    print("Generating active cells percentage plots...")
    active_df = calculate_active_percentage(df)
    
    # Save raw active cell data for reference
    active_df.to_csv(os.path.join(folders['data'], 'active_cells_data.csv'), index=False)
    
    # Call the function to plot active percentage
    active_data = plot_active_percentage(active_df, folders['active'], conditions)
    
    # 2. Create box plots for all metrics
    print("Generating box plots for all metrics...")
    for metric in metrics:
        plot_boxplots(df, metric, folders['boxplots'], conditions)
    
    # 3. Create violin plots for all metrics
    print("Generating violin plots for all metrics...")
    for metric in metrics:
        plot_violinplots(df, metric, folders['violin'], conditions)
    
    # 4. Create merged figures
    print("Generating merged comparison figures...")
    create_merged_figures(df, active_data, output_folder, conditions)
    
    # 5. Create a summary of all analyses
    print("Generating summary report...")
    create_analysis_summary(df, output_folder, folders, conditions, metrics)
    
    # Save the processed data for future reference
    df.to_csv(os.path.join(folders['data'], 'processed_data.csv'), index=False)

def create_analysis_summary(df, output_folder, folders, conditions, metrics):
    """
    Create a summary report of the analysis.
    
    Args:
        df (DataFrame): Input data
        output_folder (str): Folder to save the summary
        folders (dict): Dictionary of output folders
        conditions (list): List of conditions
        metrics (list): List of metrics analyzed
    """
    with open(os.path.join(output_folder, 'analysis_summary.txt'), 'w', encoding='utf-8') as f:
        f.write("CALCIUM IMAGING ANALYSIS SUMMARY\n")
        f.write("=" * 50 + "\n\n")
        
        # Add dataset summary
        f.write(f"Total ROIs analyzed: {len(df)}\n")
        f.write(f"Active ROIs (≥2 peaks): {df['IsActive'].sum()} ({df['IsActive'].mean()*100:.1f}%)\n")
        f.write(f"Measurements: {', '.join(map(str, sorted(df['Measurement'].unique())))}\n")
        f.write(f"Conditions: {', '.join(conditions)}\n\n")
        
        # Add information about the metrics analyzed
        f.write("Metrics analyzed:\n")
        f.write("- NumPeaks: Number of calcium transient peaks detected\n")
        f.write("- Amplitude: Mean amplitude of peaks (ΔF/F)\n")
        f.write("- Slope: Overall slope of the calcium signal\n")
        f.write("- FWHM: Full width at half maximum (peak width)\n\n")
        
        # Add information about statistical analysis
        f.write("Statistical Analysis Methods:\n")
        f.write("- Shapiro-Wilk test used to assess normality\n")
        f.write("- For normally distributed data: t-tests for pairwise comparisons\n")
        f.write("- For non-normally distributed data: Mann-Whitney U tests for pairwise comparisons\n")
        f.write("- Kruskal-Wallis test used for overall comparison when appropriate\n")
        f.write("- Significance threshold: p < 0.05\n\n")
        
        # Add information about outlier removal
        f.write("Outlier Handling:\n")
        f.write("- Outliers identified using the Interquartile Range (IQR) method\n")
        f.write("- Values outside Q1-1.5*IQR to Q3+1.5*IQR were excluded from plots and analysis\n")
        f.write("- Outlier removal was performed separately for each metric and measurement\n\n")
        
        # Add folder structure information
        f.write("Results Organization:\n")
        for name, folder in folders.items():
            f.write(f"- /{os.path.basename(folder)}/: {name.capitalize()} outputs\n")
        f.write("\n")
        
        # Add date and time of analysis
        f.write(f"Analysis performed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

def main():
    """
    Main function to run the calcium imaging analysis.
    """
    print("=" * 50)
    print("CALCIUM IMAGING DATA ANALYSIS")
    print("=" * 50)
    print("\nThis script analyzes calcium imaging data to produce publication-quality figures.")
    
    # Get input folder from user
    input_folder = input('Enter CSV folder path: ').strip()
    
    # Validate input folder
    if not os.path.isdir(input_folder):
        print(f"Error: The path '{input_folder}' is not a valid directory.")
        return
    
    # Create output folder with date
    date_str = datetime.now().strftime('%Y%m%d')
    output_folder = f"CalciumImaging_Results_{date_str}"
    os.makedirs(output_folder, exist_ok=True)
    
    print(f"\nAnalyzing data from: {input_folder}")
    print(f"Results will be saved to: {output_folder}")
    
    # Collect and process data
    print("\nCollecting data from CSV files...")
    df = collect_data(input_folder)
    
    if len(df) == 0:
        print("No valid data found. Please check your input files.")
        return
    
    # Display summary statistics
    print("\nSummary of collected data:")
    print(f"Total ROIs: {len(df)}")
    print(f"Conditions: {', '.join(sorted(df['Condition'].unique()))}")
    print(f"Measurements: {', '.join(map(str, sorted(df['Measurement'].unique())))}")
    
    # Count active cells
    active_count = df['IsActive'].sum()
    print(f"Active ROIs (≥2 peaks): {active_count} ({active_count/len(df)*100:.1f}%)")
    
    # Create all figures
    print("\nGenerating figures and statistics...")
    create_all_figures(df, output_folder)
    
    print("\nAnalysis complete!")
    print(f"All results saved to: {output_folder}")
    print("=" * 50)

# Run the script if executed directly
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print(f"\nError: {str(e)}")
        import traceback
        print(traceback.format_exc())
        print("\nPlease check your input data and try again.")

CALCIUM IMAGING DATA ANALYSIS

This script analyzes calcium imaging data to produce publication-quality figures.
Enter CSV folder path: C:\Users\joep-\Python scripts\20250319 From PC home\Calcium imaging analysis\Chemicals

Analyzing data from: C:\Users\joep-\Python scripts\20250319 From PC home\Calcium imaging analysis\Chemicals
Results will be saved to: CalciumImaging_Results_20250331

Processed 16 files with 15287 total ROIs

Summary of collected data:
Total ROIs: 15287
Conditions: Nav1.1 activator, PTZ, Veratridine, WT
Measurements: 1, 2
Active ROIs (≥2 peaks): 4028 (26.3%)

Generating figures and statistics...
Generating active cells percentage plots...


C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:642: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(
C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:642: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(


Generating box plots for all metrics...
Removed 136 outliers from NumPeaks (5.8%)


C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:830: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.boxplot(


Removed 101 outliers from NumPeaks (6.0%)


...

Removed 136 outliers from NumPeaks (5.8%)
Removed 139 outliers from Amplitude (5.9%)
Removed 61 outliers from Slope (2.6%)


C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1039: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1100: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1039: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1100: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
C:\Users\joep-\AppData\Local\Temp\ipyk

Removed 115 outliers from FWHM (4.9%)
Removed 101 outliers from NumPeaks (6.0%)
Removed 92 outliers from Amplitude (5.5%)
Removed 100 outliers from Slope (6.0%)


C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1039: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1100: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1039: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
C:\Users\joep-\AppData\Local\Temp\ipykernel_39720\1193864803.py:1100: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
C:\Users\joep-\AppData\Local\Temp\ipyk

Removed 113 outliers from FWHM (6.8%)
Generating summary report...

Analysis complete!
All results saved to: CalciumImaging_Results_20250331
